# Introduction to Text Analysis in Python

I am not an NLP person and this is outside of my expertise, but I know enough to give a basic introduction to text analysis and text processing tools.

For this tutorial + homework, I'm going to use data from Reddit. I'm getting it from PushShift, using the following code. You can run this yourself to get your own dataset (e.g., from different subreddits, or different dates). I'd recommend, however, just using the dataset that I created. I show how to import it below

In [ ]:
import requests
from datetime import datetime
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
## Code used to create the dataset - no need to run this code

endpt = 'https://api.pushshift.io/reddit/search/submission'

subreddits = ['Coronavirus', 'politics', 'aww']

# Start and end date (pushshift expects these in epoch time)
start_date = int(datetime.strptime('2021-09-11', '%Y-%m-%d').timestamp())
end_date = int(datetime.strptime('2021-09-25', '%Y-%m-%d').timestamp())


def get_posts(subreddit, before = end_date, after = start_date, result = None,  min_comments = 20):
    params = {'subreddit': subreddit,
              'num_comments': f'>{min_comments}',
              'before': before,
              'size': 500
             }
    if result == None:
        result = []
    r = requests.get(endpt, params=params)
    print(r.url)
    print(datetime.fromtimestamp(before))
    for item in r.json()['data']:
        created_time = item['created_utc']
        if created_time < after: # If we've reached the earliest we want, then return
            print(len(result))
            return result
        else:
            try:
                result.append((item['title'],item['selftext'], created_time, subreddit))
            except KeyError:
                print(item)
    time.sleep(.5)
    return get_posts(subreddit, before = created_time, result = result)


sr_data = []
for subreddit in subreddits:
    new_data = get_posts(subreddit)
    sr_data = sr_data + new_data
sr = pd.DataFrame(sr_data, columns = ['title', 'selftext', 'date', 'subreddit'])
sr.date = pd.to_datetime(sr.date, unit='s')
sr.to_csv('./sr_post_data.csv', index = False)

In [ ]:
## Code to download and import the file (DO run this code)
sr = pd.read_csv('https://github.com/jdfoote/Intro-to-Programming-and-Data-Science/blob/fall2021/resources/data/sr_post_data.csv?raw=true')

In [ ]:
# Change the date to a datetime, and put it in the index
sr.index = pd.to_datetime(sr.date)

In [ ]:
sr.head()

## Summarization

There are some simple ways to summarize text data that can be useful, without using any special NLP tools.


For example, it can be very interesting to see how the frequency of a term changes over time:

In [ ]:
# This code plots the frequency of "COVID-19", "Coronavirus", and "Trump" each day

for term in ["COVID-19", "Coronavirus", "Trump"]:
    curr_df = sr.loc[sr.title.str.contains(term) | sr.selftext.str.contains(term)]
    posts_per_day = curr_df.resample('D').size()
    posts_per_day.plot(label = term)

plt.legend()
plt.show()

### EXERCISE 1

Modify the code above to plot how often "Coronavirus" is used in each of the three subreddits over time

In [ ]:
#### YOUR CODE HERE


A similar approach is dictionary-based. The most well-known version of this is [LIWC](http://liwc.wpengine.com/), but the basic idea is that you create a set of words that are associated with a construct you are interested in, and you count how often they appear.

This is a very simple example of how you might do this to look for gendered words among our subreddits

In [ ]:
# First we change NAs and removed/deleted to empty strings
sr.loc[(pd.isna(sr.selftext)) | (sr.selftext.isin(['[removed]', '[deleted]'])), 'selftext'] = ''
sr['all_text'] = sr.title + ' ' + sr.selftext

In [ ]:
male_words = ['he', 'his']
female_words = ['she', 'hers']

# This puts all of the text of each subreddit into lists
def string_to_list(x):
    return ' '.join(x).split()
grouped_text = sr.groupby('subreddit').all_text.apply(string_to_list)

# Then, we count how often each type of words appears in each subreddit
agg = grouped_text.aggregate({'proportionMale': lambda x: sum([x.count(y) for y in male_words])/len(x),
                        'proportionFemale': lambda x: sum([x.count(y) for y in female_words])/len(x)}
                        )

In [ ]:
grouped_text

In [ ]:
agg

### EXERCISE 2

One of the trickiest parts of analysis is getting the data in the form that you want it in order to analyze/visualize it. 

I think a good visualization for this would be a barplot showing how often male and female word types appear for each subreddit. I'll give you the final call to produce the plot:

`sns.barplot(x='subreddit', y='proportion', hue = 'word_gender', data = agg_df_long)`

Now, see if you can get the data in shape so that this code actually works! :)

*Hint: You'll want to use [wide to long](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.wide_to_long.html)*

In [ ]:
## Example of how wide_to_long works (from https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.wide_to_long.html)

import numpy as np
np.random.seed(0)

df = pd.DataFrame({'A(weekly)-2010': np.random.rand(3),
                   'A(weekly)-2011': np.random.rand(3),
                   'B(weekly)-2010': np.random.rand(3),
                   'B(weekly)-2011': np.random.rand(3),
                   'X' : np.random.randint(3, size=3)})
df['id'] = df.index
df 

In [ ]:
pd.wide_to_long(df, # The data
                # The prefixes for the data columns. These will become column names that hold data values.
                stubnames = ['A(weekly)', 'B(weekly)'], 
                # i is a column which uniquely identifies each row
                i='id',
                # j is what you want to call the prefix
                j='year',
                # sep is a string that is between the stubnames and the values which will go in j
                sep='-')

In [ ]:
## Exercise 2 Code
## This code will get the df ready for pd.wide_to_long (try printing agg_df after running these to see what it looks like)
agg_df = agg.unstack(level=0)
agg_df = agg_df.reset_index()

### Your code here

In [ ]:
agg_df

In [ ]:
## Once you've created agg_df_long with the columns proportion and word_gender, you should be able to run this
sns.barplot(x='subreddit', y='proportion', hue = 'word_gender', data = agg_df_long)

### EXERCISE 3

Make your own analysis, with a different set of terms

## TF-IDF

There are more complicated approaches to summarization in Python, including using LIWC (see [here](https://pypi.org/project/liwc/)).

Almost all approaches are based on a "bag of words" approach, where the order of words is totally ignored. This is obviously a big simplification, but can often work quite well.

One thing we might want to do is to differentiate groups of texts based on how often words are used. The naive way is to just count how often words appear. However, the most common words will always appear first. So, computational linguists came up with "term frequency--inverse document frequency" (TF-IDF). This normalizes words based on how often they appear across groups of texts. A detailed explanation with code is [here](https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76).

There are a number of NLP / text analysis libraries in Python. The one I'm most familiar with is scikit-learn, which is a machine learning library. NLTK, SpaCy, and textblob are some of the most popular. Here is how to run TF-IDF in scikit-learn.

In [ ]:
## First, we prepare the data for the TF-IDF tool.
# We want each subreddit to be represented by a list of strings.
# So, we take our grouped_text (which is a list of lists of words)
# and change it into a list of three really long strings, where each
# string is all the words that appeared for that subreddit.

# This called a 'list comprehension'
as_text = [' '.join(x) for x in grouped_text]

# It is equivalent to the following for loop
as_text = []
for x in grouped_text:
    as_text.append(' '.join(x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Just gets the 5000 most common words
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

tfidf_result = vectorizer.fit_transform(as_text)
feature_names = vectorizer.get_feature_names_out()
dense = tfidf_result.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names).transpose()
df.columns = list(grouped_text.index)

In [ ]:
# This shows the values with the highest TF-IDF for r/Coronavirus
df.sort_values('Coronavirus', ascending=False).head(20)

## Relative frequency


An even simpler approach that works pretty well when comparing just two "documents" is to rank how much more often a word appears in one rather than the other.


In [ ]:
politics_str = ' '.join(sr.loc[sr.subreddit == 'politics', 'all_text']).lower()
covid_str = ' '.join(sr.loc[sr.subreddit == 'Coronavirus', 'all_text']).lower()

In [ ]:
def word_ratios(text):
    counts = {}
    tot_words = 0
    for word in text.split():
        counts[word] = counts.get(word, 0) + 1
        tot_words +=1
    result = {}
    for word, count in counts.items():
        result[word] = count/tot_words
    return result
    
    
politics_ratio = word_ratios(politics_str)
covid_ratio = word_ratios(covid_str)

In [ ]:
ratio_diff = []
for word in politics_ratio:
    if word in covid_ratio:
        ratio_diff.append((word, politics_ratio[word] - covid_ratio[word]))

In [ ]:
ratio_diff = sorted(ratio_diff, key = lambda x: x[1])

Here are the words that appear more often in r/Coronavirus

In [ ]:
ratio_diff[:15]

And here are those that appear more often in r/politics.

In [ ]:
ratio_diff[-15:][::-1] # The [::-1] just reverses the list

## Classification

Another commonly-used tool in NLP is classification. This is a "supervised machine learning" model, where you build a "training set" of items that are classified, and a machine learner uses that set to predict the classification of new items.

One very common example is sentiment. In sentiment analysis, a random set of texts is manually classified as positive, neutral, or negative. This set is then used to train a classifier to predict the sentiment of unseen texts.

It's beyond the scope of this class to learn how to do machine learning, but there are also pre-trained classifiers. One I found is from [textblob](https://textblob.readthedocs.io/en/dev/).

NLTK also has a pre-trained classifier, trained on social media data, called VADER. That is pretty similar to what we're looking at, so this example shows how to use it.

NLTK is interesting - the core is installed in Anaconda, so you should have it. However, to get various pieces to work you need to install them. So, we need to start by installing the vader lexicon.

In [ ]:
import nltk
nltk.downloader.download('vader_lexicon')

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def get_sentiment(sentence):
    vs = analyzer.polarity_scores(sentence)
    return vs['compound']

sr['sentiment'] = sr.all_text.apply(get_sentiment)

In [ ]:
sr.sort_values('sentiment', ascending=False).head()

## Topic Modeling

Finally, I'm going to show an example of topic modeling.

This is complicated, both mathematically and in code. The `gensim` package has some smart defaults, and I'm showing here the most basic, simplest way to do LDA topic modelling. There is a good, slighlty more advanced [tutorial at the gensim site](https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html).

The basic idea of topic modeling is that you are trying to optimize the likelihood of a set of distributions of words over topics and topics over documents based on the documents that actually exist. The idea is that each document can be seen as being generated by a mix of topics, and we try to find the set of topics that best matches. This works best on a large set of documents, which are themselves each quite large. In this case, the posts that we have are actually quite short. Let's plot the distribution of sizes and use topic modeling on the one with the longest posts.

In [ ]:
sr['logged_post_length'] = np.log1p(sr.all_text.str.len())

In [ ]:
sns.violinplot(data=sr, 
               y='logged_post_length',
               x='subreddit'
              );

They all look pretty similar, but r/Coronavirus is maybe a bit longer, so let's use that.

In [ ]:
# Again, we need to do an NLTK download
nltk.download('wordnet')

In [ ]:
import gensim.parsing.preprocessing as gpp
from gensim.corpora import Dictionary
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models.ldamodel import LdaModel
from pprint import pprint
from nltk.tokenize import RegexpTokenizer

def run_lda(docs, 
            n_topics, # How many topics to return
            min_count = 20 # How many docs a word must appear in to be included
           ):
    # Split the documents into tokens. This creates a list of words for each document.
    print(f"Preprocessing documents...")
    lemmatizer = WordNetLemmatizer()
    docs = [gpp.preprocess_string(x, filters=[gpp.strip_punctuation,
                                              gpp.strip_multiple_whitespaces,
                                              gpp.strip_numeric,
                                              gpp.remove_stopwords,
                                              gpp.strip_short
                                             ]) for x in docs]
    
    # Lemmatize the words

    dictionary = Dictionary(docs)
    for doc in docs:
        dictionary.add_documents([[lemmatizer.lemmatize(token) for token in doc]])
    dictionary.filter_extremes(no_below=min_count, no_above=0.5)

    corpus = [dictionary.doc2bow(doc) for doc in docs]
    
    print('Number of unique tokens: %d' % len(dictionary))
    print('Number of documents: %d' % len(corpus))
    
    
    # Train LDA model
    print("Running the model...")
    # Set training parameters.
    num_topics = n_topics
    chunksize = 2000
    passes = 20
    iterations = 400
    eval_every = None  # Don't evaluate model perplexity, takes too much time.

    # Make a index to word dictionary.
    #temp = dictionary[0]  # This is only to "load" the dictionary.
    #id2word = dictionary.id2token

    model = LdaModel(
        corpus=corpus,
        id2word=dictionary,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )
    
    return (model, corpus, dictionary, docs)
    

## r/Coronavirus topics

This code tries to look at the range of topics from the Coronavirus subreddit. It takes all of the different posts from just the Coronavirus subreddit and treats each one as a document. The problem is that they are almost all short (they are typically headlines of articles that are shared).

Note that this may take a minute to run.

In [ ]:
dataset = sr.loc[sr.subreddit == 'Coronavirus', 'all_text']
model, corpus, dictionary, docs = run_lda(dataset, 5, min_count = 10)
pprint(model.print_topics())

### Going back to the original documents to understand the topics

The most common words in a topic are helpful but they can be misleading. In order to make sure that topics are capturing something meaningful, it's important to go back to the original documents. This code shows how to do that.

First, we get the topics for each document.

In [ ]:
doc_topics = model.get_document_topics(corpus)

doc_topics[0]

This is basically a distribution of topics (by topic number) for each document. We have to do some work to connect this back to the documents themselves. First, let's put these into a dataframe.

In [ ]:
result = []
doc_id = 0
# Loop through the topic distributions for each document
for doc in doc_topics:
    # Create a temporary dictionary for this document
    curr_result = {}
    # For each topic, add an entry to the dictionary
    for topic_number, weight in doc:
        curr_result[f"topic_{topic_number}"] = weight
    # Then, add the dictionary to our list of dictionaries
    result.append(curr_result)
        
# Turn the list of dictionaries into a dataframe
doc_topic_df = pd.DataFrame(result)

Then, let's add the original documents to the dataframe. It took me a bit of wrangling to figure out how to get it in the right format.

In [ ]:
# This took some 
doc_topic_df['original_text'] = dataset.reset_index().all_text

Now, we can sort by each topic, and show the text associated.

In [ ]:
doc_topic_df.sort_values('topic_0', ascending=False).head(20)

In [ ]:
doc_topic_df.sort_values('topic_4', ascending=False).head(20)

Jupyter automatically truncates the text, but to see the full text, we can look at the document based on the index number (the far-left column in the dataframe). For example, this is document 269

In [ ]:
dataset[269]

### Visualizing results

There's a cool tool called LDAVis that previous students found, and that works well with gensim models. Here's how you run it.

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
    
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, corpus, dictionary)
pyLDAvis.display(vis)

## Finding the number of topics

One approach to choosing _k_, the number of topics, is topic coherence. Here is one example of how to calculate this for many values of k. You'd then find the max, and choose that number of topics.

In [ ]:
from gensim.models.coherencemodel import CoherenceModel
coherence = []
for k in range(5,20):
    print(f'Running with {k} topics')
    model, corpus, dictionary, docs = run_lda(dataset, n_topics=k)
    coherence_model = CoherenceModel(model=model, 
                               texts=docs, 
                               dictionary=dictionary, 
                               coherence='c_v')
    coherence.append((coherence_model.get_coherence(), k))
print(coherence)

In [ ]:
sorted(coherence, reverse=True)

### EXERCISE 4

Where topic modeling really shines is in analyzing longer texts - for example, the subreddit [changemyview](https://www.reddit.com/r/changemyview/) has fairly long posts where people explain a controversial view that they hold.

Try to figure out how to get a few hundred posts from changemyview, and run a topic model on them, where the selftext of each post is a document.